In [2]:
# add h4d proj root to sys.path so we can import stuff that's in h4d_main/h4d:
# Example: suppose you want to import something from <h4d_main>/h4d/h4d/submodule/:
# "import h4d.submodule" or "from h4d.submodule import foo"
import sys, subprocess
sys.path.append(subprocess.check_output(['git','rev-parse','--show-toplevel']).strip().decode("utf-8") )

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from collections import Counter
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import os.path
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

import io
import json
import os
from tqdm import tqdm
from PIL import Image
import xview_to_coco.wv_util as wv

In [4]:

def generate_categories():

    class_dict = {
    "Fixed-wing Aircraft": "Fixed-wing Aircraft",
    "Small Aircraft": "Fixed-wing Aircraft",
    "Cargo Plane": "Fixed-wing Aircraft",
    "Helicopter": "None",
    "Passenger Vehicle": "Passenger Vehicle",
    "Small Car": "Passenger Vehicle",
    "Bus": "Passenger Vehicle",
    "Pickup Truck": "Truck",
    "Utility Truck": "Truck",
    "Truck": "Truck",
    "Cargo Truck": "Truck",
    "Truck w/Box": "Truck",
    "Truck Tractor": "Truck",
    "Trailer": "Truck",
    "Truck w/Flatbed": "Truck",
    "Truck w/Liquid": "Truck",
    "Crane Truck": "Engineering Vehicle",
    "Railway Vehicle": "Railway Vehicle",
    "Passenger Car": "Railway Vehicle",
    "Cargo Car": "Railway Vehicle",
    "Flat Car": "Railway Vehicle",
    "Tank car": "Railway Vehicle",
    "Locomotive": "Railway Vehicle",
    "Maritime Vessel": "Maritime Vessel",
    "Motorboat": "Maritime Vessel",
    "Sailboat": "Maritime Vessel",
    "Tugboat": "Maritime Vessel",
    "Barge": "Maritime Vessel",
    "Fishing Vessel": "Maritime Vessel",
    "Ferry": "Maritime Vessel",
    "Yacht": "Maritime Vessel",
    "Container Ship": "Maritime Vessel",
    "Oil Tanker": "Maritime Vessel",
    "Engineering Vehicle": "Engineering Vehicle",
    "Tower crane": "Engineering Vehicle",
    "Container Crane": "Engineering Vehicle",
    "Reach Stacker": "Engineering Vehicle",
    "Straddle Carrier": "Engineering Vehicle",
    "Mobile Crane": "Engineering Vehicle",
    "Dump Truck": "Engineering Vehicle",
    "Haul Truck": "Engineering Vehicle",
    "Scraper/Tractor": "Engineering Vehicle",
    "Front loader/Bulldozer": "Engineering Vehicle",
    "Excavator": "Engineering Vehicle",
    "Cement Mixer": "Engineering Vehicle",
    "Ground Grader": "Engineering Vehicle",
    "Hut/Tent": "Building",
    "Shed": "Building",
    "Building": "Building",
    "Aircraft Hangar": "Building",
    "Damaged Building": "Building",
    "Facility": "Building",
    "Construction Site": "None",
    "Vehicle Lot": "None",
    "Helipad": "None",
    "Storage Tank": "None",
    "Shipping container lot": "None",
    "Shipping Container": "None",
    "Pylon": "None",
    "Tower": "None",
    }

    categories = []
    class_ids = []
    class_labels_path = "../xview_to_coco/xview_class_labels.txt"
    with open(class_labels_path) as class_labels_file:
        while True:
            line = class_labels_file.readline()
            if not line:
                break
            class_id, name = line.split(":")
            if not int(class_id) in class_ids:
                class_ids.append(int(class_id))
            name = name.split('\n')[0]
            class_entry = {}
            class_entry['supercategory'] = class_dict[name]
            class_entry['id'] = int(class_id)
            class_entry['name'] = name
            categories.append(class_entry)
    return categories, class_ids

In [5]:
xview_json = '../datasets/Xview/vanilla/xView_train.geojson'
source_imgs_dir = '../datasets/Xview/vanilla/train_images/'

In [6]:
all_coords, all_imgs, all_classes = wv.get_labels(xview_json)
print(F"{len(all_coords)}, {len(all_imgs)}, {len(all_classes)}")

100%|██████████| 601937/601937 [00:02<00:00, 234118.48it/s]

601937, 601937, 601937


In [7]:
categories_json, class_ids = generate_categories()
# print(categories_json)
catLookup = {c["id"]:c["name"] for c in categories_json}
# catLookup

In [23]:
existing_imgs = set(os.listdir(source_imgs_dir))
converted_images = set()
ann_count = 0
print_every = 10
file_index = 0
drop_counter = Counter()
not_drops = []
dcount = 0

for image in tqdm(all_imgs):
    if not image in existing_imgs:
        continue
    if image in converted_images: continue

#     arr = wv.get_image(os.path.join(source_imgs_dir, image))
    pil_img = Image.open(os.path.join(source_imgs_dir, image))
    width, height = pil_img.size
    coords = all_coords[all_imgs == image]
    classes = all_classes[all_imgs == image]

    for i in range(len(coords)):
        drop_image = False
        class_id = int(classes[i])
        if not class_id in class_ids:
            continue
        box = coords[i]
        x_min, y_min, x_max, y_max = box[0], box[1], box[2], box[3]
        assert(x_max >= x_min and y_max >= y_min)
        
        # This case never happens:
        assert(not (x_min == y_min == x_max == y_max))
        if x_min == y_min == x_max == y_max == 0:
            continue

        x,y = int(x_min), int(y_min)
        w,h = int(x_max - x_min), int(y_max - y_min)
        
        if x_min < 0 or x_min > width:
            drop_image = True
        if x_max < 0 or x_max > width:
            drop_image = True
        if y_min < 0 or y_min > height:
            drop_image = True
        if y_max < 0 or y_max > height:
            drop_image = True
        
        if drop_image:
            dcount += 1
            # Clip any bbox coords that exceed image bounds:
            x_min_cl, x_max_cl = min(max(x_min,0), width-1), min(max(x_max, 0), width-1)
            y_min_cl, y_max_cl = min(max(y_min,0), width-1), min(max(y_max, 0), height-1)
            # Calc. original bbox and clipped bbox areas:
            original_area = np.abs(x_max-x_min)*np.abs(y_max-y_min)
            clipped_area = np.abs(x_max_cl-x_min_cl)*np.abs(y_max_cl-y_min_cl)
#             print(F"i:{i}, img:{image}, cls:{classes[i]}:{catLookup[classes[i]]}, coords:{coords[i]}")
#             print("orig:", x_min, x_max, y_min, y_max, original_area)
#             print("new :", x_min_cl, x_max_cl, y_min_cl, y_max_cl, clipped_area)
#             print(F"cliped/original area ratio: {clipped_area/original_area:.2f}")
            # If the truncated bbox area is less than 2/3 the full bbox area, drop it:
            if clipped_area/original_area < 0.666666:
                drop_counter[catLookup[int(classes[i])]] += 1
            else:
                not_drops.append({
                    "i": i
                    , "img": image
                    , "coords": [x_min, x_max, y_min, y_max]
                    , "coords_clipped": [x_min_cl, x_max_cl, y_min_cl, y_max_cl]
                })
            continue
        ann_count += 1

    file_index += 1
    converted_images.add(image)


100%|██████████| 601937/601937 [01:09<00:00, 8699.20it/s]


In [24]:
print("ann_count: ", ann_count)
# These are the ones we drop, even with 
print("total dropped ann's: ", sum(drop_counter.values()))
# print(drop_counter)
print("dcount: ", dcount)
# These are the ones we keep with the 2/3 area filter:
print("no_drops: ", len(not_drops))
print(dcount - (len(not_drops) + sum(drop_counter.values())))

ann_count:  591810
total dropped ann's:  5780
dcount:  9917
no_drops:  4137
0


In [25]:
print(not_drops[:5])

[{'i': 0, 'img': '2355.tif', 'coords': [2712.0, 2746.0, 1145.0, 1177.0], 'coords_clipped': [2712.0, 2738, 1145.0, 1177.0]}, {'i': 2, 'img': '2355.tif', 'coords': [2687.0, 2740.0, 1338.0, 1399.0], 'coords_clipped': [2687.0, 2738, 1338.0, 1399.0]}, {'i': 6, 'img': '2355.tif', 'coords': [2709.0, 2747.0, 1014.0, 1072.0], 'coords_clipped': [2709.0, 2738, 1014.0, 1072.0]}, {'i': 9, 'img': '2355.tif', 'coords': [2704.0, 2749.0, 2123.0, 2151.0], 'coords_clipped': [2704.0, 2738, 2123.0, 2151.0]}, {'i': 12, 'img': '2355.tif', 'coords': [2703.0, 2745.0, 2007.0, 2040.0], 'coords_clipped': [2703.0, 2738, 2007.0, 2040.0]}]
